# <center>**Podstawy Sztucznej Inteligencji**</center>
## <center>Projekt 2</center>
### <center> Drzewa decyzyjne oraz sieci neuronowe przy predykcji spalonych obszarów lasu</center>



<center>Adrian Nadratowski</center>
<center>Marcin Skrzypkowski</center>

Celem projektu jest stworzenie modeli przewidujących spalony obszar lasu przy wykorzystaniu udostępnionych nam zestawów danych, pobranych z __Machine Learning Repository__ (https://archive.ics.uci.edu/ml/datasets/Forest+Fires). Naszym zadaniem było przeanalizować powyższe dane, a następnie zaprojektować, nauczyć i zwalidować dwa regresory: model sieci neuronowej oraz drzewa decyzyjnego. Sprawdzenie miało odbyć się z pomocą _k_-krotnej walidacji krzyżowej.

Biblioteki wykorzystane do projektu:
  * pandas
  * tensorflow
  * scikit-learn
  * matplotlib

# %TODO:
### do uzupełnienia:
  * kluczowe decyzje projektowe
  * opis struktury programu
  * instrukcja użytkownika (+opis interfejsu)
  * wkład poszczególnych autorów
  * analiza zbioru danych (również odnośnie brakujących wartości)
  

### Opis struktury programu

Projekt podzielony został na __X__ części. W pierwszej następuje analiza danych oraz sprawdzenie czy nie brajkuje jakichś wartości, które mogłyby wpłynąć na jakość uczenia. Następnie przedstawiona została implementacja drzew decyzyjnych oraz walidacja kilku modeli o różnych parametrach. Kolejnym punktem jest sieć neuronowa oraz jej walidacja. Na sam koniec obydwa sposoby modelowania zostały ze sobą porównane. 

### Instrukcja obsługi użytkownika

  * po uruchomieniu niniejszego projektu należy upewnić się, czy zainstalowane są wszystkie wymagane biblioteki wymienione we wprowadzeniu
  * w folderze _../data_ powinien znajdować się zbiór danych, na którym operować będzie poniższy skrypt
  * projekt składa się z __4__ głównych części: 
    1. import i walidacja danych
    2. model drzewa decyzyjnego
    3. model sieci neuronowej
    4. podsumowanie
    i w tej kolejności należy uruchamiać odpowiednie skrypty
  * każda część projektu uruchamiana jest kolejno od najwyższych do najniższych komórek
  * każda komórka jest szczegółowo opisana z wyjaśnieniem procesu w niej zaimplementowanego